In [70]:
import pandas as pd

In [71]:
import geopandas as gpd

In [72]:
poverty = pd.read_csv(r"C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\rough_poverty\poverty.csv")

In [73]:
poverty.head()

,geo_id,state_name,county,year,total_population,population_below_poverty,percent_below_poverty,adults(25+)_with_high_school,percent_adults(25+)_with_high_school,adults(25+)_with_college_degree,percent_adults(25+)_with_college_degree
0,0500000US01007,Alabama,Bibb,2015,20767,3488,16.8,5960,13.5,1657,3.7
1,0500000US01009,Alabama,Blount,2015,57126,9552,16.7,13032,13.1,5060,2.4
2,0500000US01015,Alabama,Calhoun,2015,113602,23325,20.5,24769,15.3,13756,4.2
3,0500000US01017,Alabama,Chambers,2015,33687,7273,21.6,8996,15.7,2750,4.0
4,0500000US01019,Alabama,Cherokee,2015,25575,4918,19.2,6693,18.5,2562,4.5


In [74]:
# Read the shapefile
gdf = gpd.read_file('C:/Users/athar/OneDrive/Desktop/fulton_ring/cb_2018_us_county_5m/cb_2018_us_county_5m.shp')

In [75]:
print(gdf.head())

  STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID      NAME LSAD  \
0      39      071  01074048  0500000US39071  39071  Highland   06   
1      06      003  01675840  0500000US06003  06003    Alpine   06   
2      12      033  00295737  0500000US12033  12033  Escambia   06   
3      17      101  00424252  0500000US17101  17101  Lawrence   06   
4      28      153  00695797  0500000US28153  28153     Wayne   06   

        ALAND     AWATER                                           geometry  
0  1432479992   12194983  POLYGON ((-83.86976 39.05553, -83.86568 39.247...  
1  1912292630   12557304  POLYGON ((-120.07248 38.50987, -120.07239 38.7...  
2  1701544502  563927612  POLYGON ((-87.62999 30.87766, -87.62946 30.880...  
3   963936864    5077783  POLYGON ((-87.91028 38.57493, -87.90811 38.850...  
4  2099745573    7255476  POLYGON ((-88.94317 31.78421, -88.94336 31.824...  


In [76]:
# keep only the join key + geometry
gdf_slim = gdf[["GEOID", "geometry"]].copy()
gdf_slim["GEOID"] = gdf_slim["GEOID"].astype(str).str.zfill(5)

In [77]:
# create a FIPS code column (last 5 characters of GEO_ID)
poverty["county_fips"] = poverty["geo_id"].str[-5:]

In [78]:
# join on FIPS = GEOID
merged = poverty.merge(
    gdf_slim.rename(columns={"GEOID": "county_fips"}),
    left_on="county_fips",
    right_on="county_fips",
    how="left"
)

In [79]:
# Convert merged DataFrame into a GeoDataFrame
gdf = gpd.GeoDataFrame(merged, geometry="geometry", crs="EPSG:4326")

In [80]:
merged.head()

,geo_id,state_name,county,year,total_population,population_below_poverty,percent_below_poverty,adults(25+)_with_high_school,percent_adults(25+)_with_high_school,adults(25+)_with_college_degree,percent_adults(25+)_with_college_degree,county_fips,geometry
0,0500000US01007,Alabama,Bibb,2015,20767,3488,16.8,5960,13.5,1657,3.7,01007,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
1,0500000US01009,Alabama,Blount,2015,57126,9552,16.7,13032,13.1,5060,2.4,01009,"POLYGON ((-86.96336 33.85822, -86.9202 33.8734..."
2,0500000US01015,Alabama,Calhoun,2015,113602,23325,20.5,24769,15.3,13756,4.2,01015,"POLYGON ((-86.14371 33.70913, -86.12388 33.710..."
3,0500000US01017,Alabama,Chambers,2015,33687,7273,21.6,8996,15.7,2750,4.0,01017,"POLYGON ((-85.59318 33.10734, -85.23244 33.108..."
4,0500000US01019,Alabama,Cherokee,2015,25575,4918,19.2,6693,18.5,2562,4.5,01019,"POLYGON ((-85.84384 34.24345, -85.82548 34.240..."


In [81]:
merged.to_csv('./final_poverty.csv', index=False)

In [82]:
# Use a raw string to avoid unicode escape errors in the file path
gdf.to_file(r'C:\Users\athar\OneDrive\Desktop\fulton_ring\fahe\poverty.geojson', driver='GeoJSON')